# Create a drumkit using found samples, ipytone, and jupyter widgets

- In this overview we will import samples from the web and use ipytone and jupyter widgets to play them with a crontroller

In [63]:
import os
import glob
import ipytone
from ipywidgets import widgets
import pandas as pd
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt

# Download the data
- download the data found at this link and put it in the same directory that you have this file

In [12]:
sample_dir_808 = "Roland TR-808/*/*.WAV"
sample_files_808 = glob.glob(sample_dir_808)

In [14]:
# sample_files_808

In [28]:
sample_codes_808 = """bass_drum BD
snare_drum SD
low_tom LT
mid_tom MT
hi_tom HT
low_conga LC
mid conga MC
hi_onga HC
rim_shot RS
claves CL
hand_clap CP
maracas MA
cowbell CB
cymbal CY
open_hi_hat OH
closed_hi_hat CH"""

In [82]:
sample_lables = [list(reversed(i.split())) for i in sample_codes_808.split('\n')]

In [83]:
sample_lable_dict = dict()
for i in sample_lables:
    sample_lable_dict[i[0]] = i[1]


In [84]:
sample_lable_dict

{'BD': 'bass_drum',
 'SD': 'snare_drum',
 'LT': 'low_tom',
 'MT': 'mid_tom',
 'HT': 'hi_tom',
 'LC': 'low_conga',
 'MC': 'conga',
 'HC': 'hi_onga',
 'RS': 'rim_shot',
 'CL': 'claves',
 'CP': 'hand_clap',
 'MA': 'maracas',
 'CB': 'cowbell',
 'CY': 'cymbal',
 'OH': 'open_hi_hat',
 'CH': 'closed_hi_hat'}

In [85]:
sample_lable_df = pd.DataFrame([sample_lable_dict.keys(), sample_lable_dict.values()]).T

In [86]:
sample_names = [i.split('\\')[-1].split('.')[0] for i in sample_files_808]

In [87]:
sample_df = pd.DataFrame(sample_names, columns=['sample_name'])

In [88]:
def lable_check(sample_name):
    mask = [i in sample_name for i in sample_lable_dict.keys()]
    key = sample_lable_df.loc[mask][0].values[0]
    return sample_lable_dict[key]

sample_df['lable'] = sample_df.sample_name.apply(lable_check)

In [89]:
sample_df

,sample_name,lable
0,BD0000,bass_drum
1,BD0010,bass_drum
2,BD0025,bass_drum
3,BD0050,bass_drum
4,BD0075,bass_drum
...,...,...
111,SD7500,snare_drum
112,SD7510,snare_drum
113,SD7525,snare_drum
114,SD7550,snare_drum


In [90]:
sample_df['file_name'] = sample_files_808

In [91]:
sample_df

,sample_name,lable,file_name
0,BD0000,bass_drum,Roland TR-808\BD\BD0000.WAV
1,BD0010,bass_drum,Roland TR-808\BD\BD0010.WAV
2,BD0025,bass_drum,Roland TR-808\BD\BD0025.WAV
3,BD0050,bass_drum,Roland TR-808\BD\BD0050.WAV
4,BD0075,bass_drum,Roland TR-808\BD\BD0075.WAV
...,...,...,...
111,SD7500,snare_drum,Roland TR-808\SD\SD7500.WAV
112,SD7510,snare_drum,Roland TR-808\SD\SD7510.WAV
113,SD7525,snare_drum,Roland TR-808\SD\SD7525.WAV
114,SD7550,snare_drum,Roland TR-808\SD\SD7550.WAV


In [92]:
wav_df = sample_df.file_name.apply(lambda x: wavfile.read(x)[1])


C:\Users\smada\AppData\Local\Temp\ipykernel_12972\3167456665.py:1: WavFileWarning: Reached EOF prematurely; finished at 264646 bytes, expected 264647 bytes from header.
  wav_df = sample_df.file_name.apply(lambda x: wavfile.read(x)[1])
C:\Users\smada\AppData\Local\Temp\ipykernel_12972\3167456665.py:1: WavFileWarning: Reached EOF prematurely; finished at 44146 bytes, expected 44147 bytes from header.
  wav_df = sample_df.file_name.apply(lambda x: wavfile.read(x)[1])
C:\Users\smada\AppData\Local\Temp\ipykernel_12972\3167456665.py:1: WavFileWarning: Reached EOF prematurely; finished at 132346 bytes, expected 132347 bytes from header.
  wav_df = sample_df.file_name.apply(lambda x: wavfile.read(x)[1])
C:\Users\smada\AppData\Local\Temp\ipykernel_12972\3167456665.py:1: WavFileWarning: Reached EOF prematurely; finished at 176446 bytes, expected 176447 bytes from header.
  wav_df = sample_df.file_name.apply(lambda x: wavfile.read(x)[1])
C:\Users\smada\AppData\Local\Temp\ipykernel_12972\31674566

In [94]:
sample_df["signal"] = wav_df

In [105]:
sum(sample_df.signal.apply(len) == 0)

0

In [107]:
# wavfile.read(sample_df.file_name[1])

# Read up on the scipy wavefile function
- We were able to read all of the files into the array but we got some errors due to mangled file headers
- https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.wavfile.read.html
- some information about the issue above can be found in this issue: https://github.com/scipy/scipy/issues/6700#issuecomment-255592052

In [113]:
audio_buffers = sample_df.signal.apply(lambda x: ipytone.AudioBuffer(x.astype(float))).to_list()

In [118]:
audio_buffers_dict = dict(zip(sample_df.sample_name.to_list(), audio_buffers))

In [119]:
sample_808 = ipytone.AudioBuffers(audio_buffers_dict)

In [121]:
sample_df.sample_name[5]

'BD1000'

In [122]:
sample_808.buffers["BD1000"]

AudioBuffer(loaded=True, duration=0.22970833333333332)

In [123]:
sample_808_players = ipytone.Players(sample_808.buffers).to_destination()

In [125]:
sample_808_players.get_player("BD1000").start()

Player()

In [128]:
@widgets.interact(x = sample_df.sample_name)
def plot_sound_file(x=sample_df.sample_name[0]):
    plt.plot(sample_808.buffers[x].array)
    sample_808_players.get_player(x).start().stop("+1")

interactive(children=(Dropdown(description='x', options=('BD0000', 'BD0010', 'BD0025', 'BD0050', 'BD0075', 'BD…

In [132]:
def create_pad():
    pad_layout = widgets.Layout(height='100px', width='100px')

    play_sample_button = widgets.Button(description=sample_df.sample_name[0], layout=pad_layout)
    
    def play_sample(b):
        sample_808_players.get_player(play_sample_button.description).start()
    
    play_sample_button.on_click(play_sample)
    
    
    sample_dropdown = widgets.Dropdown(description="",
                                       options=sample_df.sample_name,
                                       value=sample_df.sample_name[0],
                                       layout=widgets.Layout(width="100px")
                                      )
    
    def change_sample_name(change):
        if change['new']:
            play_sample_button.description = sample_dropdown.value
        else:
            pass
    
    sample_dropdown.observe(change_sample_name, names='value')
    
    vbox = widgets.VBox([play_sample_button, sample_dropdown])
    
    return vbox

pad_one = create_pad()
pad_one


In [133]:
pads = [create_pad() for i in range(16)]

# Create a standard rack of drum machine pads
pad_rack = widgets.VBox([widgets.HBox(pads[:4]),
                         widgets.HBox(pads[4:8]),
                         widgets.HBox(pads[8:12]),
                         widgets.HBox(pads[12:16])
                        ])
pad_rack

---
# Playing samples using the controller

In [134]:
controller = widgets.Controller()

In [135]:
controller

Controller()

In [136]:
# Mapping controller buttons to the drumpads
# There as to be a better way to do this but right now this is what works

def button_sample_hit(change):
    if change['new']:
        sample_808_players.get_player(pads[0].children[0].description).start().stop("+1")
    else:
        pass

controller.buttons[0].observe(button_sample_hit, names='value')


def button_sample_hit(change):
    if change['new']:
        sample_808_players.get_player(pads[1].children[0].description).start().stop("+1")
    else:
        pass

controller.buttons[1].observe(button_sample_hit, names='value')

def button_sample_hit(change):
    if change['new']:
        sample_808_players.get_player(pads[2].children[0].description).start().stop("+1")
    else:
        pass

controller.buttons[2].observe(button_sample_hit, names='value')


def button_sample_hit(change):
    if change['new']:
        sample_808_players.get_player(pads[3].children[0].description).start().stop("+1")
    else:
        pass

controller.buttons[3].observe(button_sample_hit, names='value')

# The D-pad
def button_sample_hit(change):
    if change['new']:
        sample_808_players.get_player(pads[12].children[0].description).start().stop("+1")
    else:
        pass

controller.buttons[12].observe(button_sample_hit, names='value')

def button_sample_hit(change):
    if change['new']:
        sample_808_players.get_player(pads[13].children[0].description).start().stop("+1")
    else:
        pass

controller.buttons[13].observe(button_sample_hit, names='value')

def button_sample_hit(change):
    if change['new']:
        sample_808_players.get_player(pads[14].children[0].description).start().stop("+1")
    else:
        pass

controller.buttons[14].observe(button_sample_hit, names='value')

def button_sample_hit(change):
    if change['new']:
        sample_808_players.get_player(pads[15].children[0].description).start().stop("+1")
    else:
        pass

controller.buttons[15].observe(button_sample_hit, names='value')

In [139]:
# widgets.VBox([widgets.HBox(controller.buttons[:4]), widgets.HBox(pads[:4])])

# Main Button HUD

In [138]:
buttons_pads = np.column_stack([controller.buttons[:4], pads[:4]])

buttons_pads_disp = [widgets.VBox([widgets.HBox(buttons_pads.tolist()[i])]) for i in range(len(buttons_pads))]

widgets.HBox(buttons_pads_disp)

# DPAD HUD 

In [142]:
buttons_pads_dpad = np.column_stack([controller.buttons[12:16], pads[12:16]])

buttons_pads__dpad_disp = [widgets.VBox([widgets.HBox(buttons_pads_dpad.tolist()[i])]) for i in range(len(buttons_pads_dpad))]

widgets.HBox(buttons_pads__dpad_disp)

# Conclusion
- We were able to leverage several tools to import sample data from the web catagorize it link it to a drum pad that can be used in multiple formats one of which is the common game controller